In [1]:
import numpy as np
import pandas as pd
import math
import scipy
import sys
from tqdm import tqdm
import functools
import networkx as nx
import itertools

import portion as P
import re
import datetime
import queue
import sympy as sym

# interval = P.empty()
# for i, j in [(0, 1), (2, 4), (6, 7), (3, 5)]:
#     interval = interval | P.closed(i, j)
# print(interval)

In [2]:
scipy.signal.correlate2d(np.array([
    [1,2,0],
    [1,0,0],
    [1,0,0]]), np.array([[1,1,1],
                         [1,1,0],
                         [1,0,0]]), mode='same')

array([[1, 4, 2],
       [4, 5, 2],
       [2, 2, 0]])

In [343]:
class Board():
    def __init__(self, map) -> None:
        self.map = np.pad(map,1,constant_values=3)
        
        j = 0
        while map[1,j] == 3:
            j += 1
        self.location = (1,j+1)
        self.facing = 0

    def next_position(self):
        """find next position"""
        i,j = self.location
        if self.facing == 0:
            #move right
            if self.map[i,j+1] == 0:
                return (i,j+1)
            if self.map[i,j+1] == 9:
                return None
            if self.map[i,j+1] == 3:
                #wrap around
                j = 0
                # print(f'T {self.map[i,j]}')
                while self.map[i,j] == 3:
                    j += 1
                    # print(f'TT {self.map[i,j]}')
                if self.map[i,j] == 9:
                    return None
                else:
                    return (i,j)
                
        elif self.facing == 1:
            #move down
            if self.map[i+1,j] == 0:
                return (i+1,j)
            if self.map[i+1,j] == 9:
                return None
            if self.map[i+1,j] == 3:
                #wrap around
                i = 0
                while self.map[i,j] == 3:
                    i += 1
                if self.map[i,j] == 9:
                    return None
                else:
                    return (i,j)

        elif self.facing == 2:
            #move left
            if self.map[i,j-1] == 0:
                return (i,j-1)
            if self.map[i,j-1] == 9:
                return None
            if self.map[i,j-1] == 3:
                #wrap around
                j = self.map.shape[1]-1
                # print(f'T {self.map[i,j]}')
                while self.map[i,j] == 3:
                    j -= 1
                    # print(f'TT {self.map[i,j]}')
                if self.map[i,j] == 9:
                    return None
                else:
                    return (i,j)

        elif self.facing == 3:
            #move up
            if self.map[i-1,j] == 0:
                return (i-1,j)
            if self.map[i-1,j] == 9:
                return None
            if self.map[i-1,j] == 3:
                #wrap around
                i = self.map.shape[0]-1
                while self.map[i,j] == 3:
                    i -= 1
                if self.map[i,j] == 9:
                    return None
                else:
                    return (i,j)

    def move(self, steps:int):
        for _ in range(steps):
            new_location = self.next_position()
            if new_location:
                # print(f'moved!')
                self.location = new_location
            else:
                # print(f'stoped!')
                break

    def rotate(self, direction:str):
        if direction == 'L':
            self.facing = (self.facing -1) %4
        elif direction == 'R':
            self.facing = (self.facing +1) %4
        else:
            print(f'bad rotate input: {direction}')
        # print(self.facing)
    
    def __repr__(self) -> str:
        tr = {
            3: ' ',
            0: '.',
            9: '#'
        }
        ret = ''
        ii,jj = self.location
        for i in range(self.map.shape[0]):
            for j in range(self.map.shape[1]):
                if ii==i and jj==j:
                    ret += 'O'
                else:
                    ret += tr[self.map[i,j]]
            ret += '\n'
        return ret

In [227]:
file1 = open('22.txt', 'r')
lines = file1.readlines()

input_map = []
path = ''

for l in lines:
    if len(l.replace('\n','')) == 0:
        break
    input_map.append(l.replace('\n',''))
    # print()
path = re.findall(r'(\d+|[A-Z])',lines[-1])

# print(path)

max_width = max([len(x) for x in input_map])

# print(input_map)

np_translate = {
    ' ': 3,
    '.': 0,
    '#': 9
}

input_np = []
for x in input_map:

    row = x.ljust(max_width,' ')
    input_np.append([np_translate[c] for c in row])

input_np = np.array(input_np)
# input_np


In [228]:
b=Board(input_np)

In [117]:
# path

In [229]:
for p in path:
    if p.isdigit():
        b.move(int(p))
    else:
        b.rotate(p)
b.location

(88, 67)

In [ ]:
(88, 67)

In [230]:
1000*b.location[0] + 4*b.location[1] + b.facing

88268

# Second part

In [234]:
##total number of steps
sum([int(p) for p in path if p.isdigit()])

50882

In [634]:
##update input's format

file1 = open('22.txt', 'r')
lines = file1.readlines()

input_map = []
path = ''

for l in lines:
    if len(l.replace('\n','')) == 0:
        break
    input_map.append(l.replace('\n',''))
    # print()
path = re.findall(r'(\d+|[A-Z])',lines[-1])

# print(path)

max_width = max([len(x) for x in input_map])

input_map = [x.ljust(max_width,' ') for x in input_map]
# input_map



In [639]:
np_input = np.array([[x for x in l] for l in input_map])
square_len = int(np_input.shape[1]/3)

##move 2
sub_from = np_input[0:square_len,2*square_len:3*square_len]
sub_to = np_input[square_len:2*square_len,2*square_len:3*square_len]
# sub_to = np.rot90(sub_from, k=1)
# sub_from= ' '
np_input[square_len:2*square_len,2*square_len:3*square_len] = np.rot90(sub_from, k=1)
np_input[0:square_len,2*square_len:3*square_len] = ' '

##move 5
sub_from = np_input[2*square_len:3*square_len,0:square_len]
sub_to = np_input[square_len:2*square_len,0:square_len]
# sub_to = np.rot90(sub_from, k=1)
# sub_from= ' '
np_input[square_len:2*square_len,0:square_len] = np.rot90(sub_from, k=1)
np_input[2*square_len:3*square_len,0:square_len] = ' '

##move 6
sub_from = np_input[3*square_len:4*square_len,0:square_len]
sub_to = np_input[3*square_len:4*square_len,square_len:2*square_len]
# sub_to = np.rot90(sub_from, k=3)
# sub_from= ' '
np_input[3*square_len:4*square_len,square_len:2*square_len] = np.rot90(sub_from, k=3)
np_input[3*square_len:4*square_len,0:square_len] = ' '

ret = ''
for l in np_input:
    for x in l:
        ret += x
    ret += '\n'

# print(ret)

In [648]:
class Board2(Board):
    def get_section_loc(self,i=None,j=None):
        if i == None:
            i,j = self.location

        section_len = int((self.map.shape[1]-2)/3)
        return (i-1)//section_len, (j-1)//section_len
    
    def get_loc_within_section(self,i=None,j=None):
        if i == None:
            i,j = self.location

        section_len = int((self.map.shape[1]-2)/3)
        return (i-1)%section_len, (j-1)%section_len

    def at_section_edge(self,i=None,j=None):
        ##finds if we're at section edge (up,right,down,left)
        if i == None:
            i,j = self.location
        section_len = int((self.map.shape[1]-2)/3)

        i = (i-1)%section_len
        j = (j-1)%section_len
        return i == 0, j == section_len-1, i == section_len-1, j == 0

    def next_position(self,i=None,j=None):
        """find next position"""
        if i == None:
            i,j = self.location
        section_len = int((self.map.shape[1]-2)/3)
        section_loc = self.get_section_loc(i,j)
        loc_in_section = self.get_loc_within_section(i,j)
        e1, e2, e3,e4 = self.at_section_edge(i,j)
        

        if self.facing == 0:
            #move right
            if not e2 or (section_loc[0] == 1 and section_loc[1] == 1):
                if self.map[i,j+1] == 0:
                    return (i,j+1)
                if self.map[i,j+1] == 9:
                    return None
            else:
                if section_loc[0] == 0:
                    diag_len = section_len-loc_in_section[0]
                    next_pos = (i+diag_len,j+diag_len)
                    if self.map[next_pos] == 0:
                        self.rotate('R')
                        return (next_pos)
                    else:
                        return None
                elif section_loc[0] == 2:
                    diag_len = loc_in_section[0]+1
                    next_pos = (j,i)
                    if self.map[next_pos] == 0:
                        self.rotate('L')
                        return (next_pos)
                    else:
                        return None
                elif section_loc[0] == 1:
                    next_pos = ((section_len-loc_in_section[0])+3*section_len,j-section_len)
                    if self.map[next_pos] == 0:
                        self.rotate('R')
                        self.rotate('R')
                        return (next_pos)
                    else:
                        return None
                elif section_loc[0] == 3:
                    next_pos = ((section_len-loc_in_section[0])+section_len,j+section_len)
                    if self.map[next_pos] == 0:
                        self.rotate('R')
                        self.rotate('R')
                        return (next_pos)
                    else:
                        return None
            
                
        elif self.facing == 1:
            #move down
            if not e3 or (section_loc[0] in [0,1,2] and section_loc[1] == 1):
                # print('moving normally')
                if self.map[i+1,j] == 0:
                    return (i+1,j)
                if self.map[i+1,j] == 9:
                    return None
            else:
                if section_loc[1] == 0:
                    diag_len = section_len-loc_in_section[1]
                    next_pos = (i+diag_len,j+diag_len)
                    if self.map[next_pos] == 0:
                        self.rotate('L')
                        return (next_pos)
                    else:
                        return None
                elif section_loc[1] == 2:
                    next_pos = (j,i)
                    if self.map[next_pos] == 0:
                        self.rotate('R')
                        return (next_pos)
                    else:
                        return None
                elif section_loc[1] == 1:
                    next_pos = (1,j)
                    if self.map[next_pos] == 0:
                        return (next_pos)
                    else:
                        return None

        elif self.facing == 2:
            #move left
            if not e4 or (section_loc[0] == 1 and section_loc[1] == 1):
                if self.map[i,j-1] == 0:
                    return (i,j-1)
                if self.map[i,j-1] == 9:
                    return None
            else:
                if section_loc[0] == 0:
                    diag_len = section_len-loc_in_section[0]
                    next_pos = (i+diag_len,j-diag_len)
                    if self.map[next_pos] == 0:
                        self.rotate('L')
                        return (next_pos)
                    else:
                        return None
                elif section_loc[0] == 2:
                    diag_len = loc_in_section[0]+1
                    next_pos = (i-diag_len,j-diag_len)
                    if self.map[next_pos] == 0:
                        self.rotate('R')
                        return (next_pos)
                    else:
                        return None
                elif section_loc[0] == 1:
                    next_pos = ((section_len-loc_in_section[0])+3*section_len,j+section_len)
                    # print(next_pos)
                    if self.map[next_pos] == 0:
                        self.rotate('R')
                        self.rotate('R')
                        return (next_pos)
                    else:
                        return None
                elif section_loc[0] == 3:
                    next_pos = ((section_len-loc_in_section[0])+section_len,j-section_len)
                    if self.map[next_pos] == 0:
                        self.rotate('R')
                        self.rotate('R')
                        return (next_pos)
                    else:
                        return None

        elif self.facing == 3:
            #move up
            if not e1 or (section_loc[0] in [1,2,3] and section_loc[1] == 1):
                # print('moving normally')
                if self.map[i-1,j] == 0:
                    return (i-1,j)
                if self.map[i-1,j] == 9:
                    return None
            else:
                if section_loc[1] == 0:
                    diag_len = section_len-loc_in_section[1]
                    next_pos = (i-diag_len,j+diag_len)
                    if self.map[next_pos] == 0:
                        self.rotate('R')
                        return (next_pos)
                    else:
                        return None
                elif section_loc[1] == 2:
                    diag_len = loc_in_section[1]+1
                    next_pos = (i-diag_len,j-diag_len)
                    if self.map[next_pos] == 0:
                        self.rotate('L')
                        return (next_pos)
                    else:
                        return None
                elif section_loc[1] == 1:
                    next_pos = (4*section_len,j)
                    if self.map[next_pos] == 0:
                        return (next_pos)
                    else:
                        return None

    def move(self, steps:int):
        for _ in range(steps):
            new_location = self.next_position()
            if new_location:
                # print(f'moved!')
                self.location = new_location
            else:
                # print(f'stoped!')
                break

b = Board2(input_np)

In [649]:


input_np = []
for row in np_input:
    input_np.append([np_translate[c] for c in row])

input_np = np.array(input_np)

b = Board2(input_np)

In [642]:
b

                                                                                                                                                        
                                                   O.....#...........#..............#.......#.#......                                                   
                                                   ..........#...........#.......#..........#...#....                                                   
                                                   ..........#..................#.................#.#                                                   
                                                   .....#.........................................##.                                                   
                                                   ....#............#.......#............#..........#                                                   
                                                   ...#...........................

In [401]:
b.location

(1, 5)

In [534]:
b = Board2(input_np)
b.facing = 3

print(b.next_position(1,6))
print(b.facing)

(16, 6)
3


In [501]:
b.at_section_edge(1,8)

(True, True, False, False)

In [502]:
b.get_section_loc(1,8)

(0, 1)

In [427]:
b.get_loc_within_section(9,12)

(0, 3)

In [645]:
path[:10]

['39', 'L', '35', 'R', '25', 'L', '7', 'R', '47', 'L']

In [650]:
b = Board2(input_np)


for p in path:
    if p.isdigit():
        b.move(int(p))
    else:
        b.rotate(p)
b.location

(58, 45)

In [558]:
b.map[b.location] = 2

b

              
     ...#     
     .#..     
     #...     
     ....     
 ..O....#.... 
 ....#......# 
 ...#.....#.. 
 ......#..... 
     ...#     
     ....     
     .#..     
     ....     
     ....     
     .#..     
     ....     
     #...     
              

In [658]:
b_map_for_np = []
tr = {
    3: ' ',
    0: '.',
    9: '#',
    10: '>',
    11: 'v',
    12: '<',
    13: '^',
}

b.map[b.location] = 10+b.facing
tmp = b.map.copy()[1:-1,1:-1]

for row in tmp:
    r = []
    for el in row:
        r.append(tr[el])
    b_map_for_np.append(r)

In [659]:
##revert transformations

# np_input = b.map.copy()
np_input = np.array(b_map_for_np)

##revert move 2
sub_from = np_input[0:square_len,2*square_len:3*square_len]
sub_to = np_input[square_len:2*square_len,2*square_len:3*square_len]
# sub_to = np.rot90(sub_from, k=1)
# sub_from= ' '
np_input[0:square_len,2*square_len:3*square_len] = np.rot90(sub_to, k=3)
np_input[square_len:2*square_len,2*square_len:3*square_len] = ' '

##revert move 5
sub_from = np_input[2*square_len:3*square_len,0:square_len]
sub_to = np_input[square_len:2*square_len,0:square_len]
# sub_to = np.rot90(sub_from, k=1)
# sub_from= ' '
np_input[2*square_len:3*square_len,0:square_len] = np.rot90(sub_to, k=3)
np_input[square_len:2*square_len,0:square_len] = ' '

##revert move 6
sub_from = np_input[3*square_len:4*square_len,0:square_len]
sub_to = np_input[3*square_len:4*square_len,square_len:2*square_len]
# sub_to = np.rot90(sub_from, k=3)
# sub_from= ' '
np_input[3*square_len:4*square_len,0:square_len] = np.rot90(sub_to, k=1)
np_input[3*square_len:4*square_len,square_len:2*square_len] = ' '

np_input

ret = ''
for l in np_input:
    for x in l:
        ret += x
    ret += '\n'

print(ret)


                                                  ......#...........#..............#.......#.#..............#...................#...#.................
                                                  ..........#...........#.......#..........#...#.............#..#..#.#....#......................#....
                                                  ..........#..................#.................#.#............##.......#..........#......#.......#.#
                                                  .....#.........................................##.................#..........##.....................
                                                  ....#............#.......#............#..........#..#.....#......#...................#.#.#.....#....
                                                  ...#.............................#.#..#..........#..#..............#.........#......................
                                                  .........#.#...##......#........#...........

In [660]:
ans2 = None
for i,row in enumerate(np_input):
    for j,el in enumerate(row):
        if el in ['>','v','<','^']:
            ans2 = (i+1,j+1,['>','v','<','^'].index(el))
ans2

(145, 43, 1)

In [661]:
1000*ans2[0] + 4*ans2[1] + ans2[2]

145173

In [ ]:
146169 #too high
145173 #too high